<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:

*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached soupsieve-2.8-py3-none-any.whl (36 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [beautifulsoup4]


In [2]:
# ── Imports ──
import sys
import re
import unicodedata

import requests
from bs4 import BeautifulSoup
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
# ── Extract date and time from table cell ──
def date_time(table_cells):
    """
    Returns the date and time from the HTML table cell.
    Input: element of a table data cell.
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


# ── Extract booster version from table cell ──
def booster_version(table_cells):
    """
    Returns the booster version from the HTML table cell.
    Input: element of a table data cell.
    """
    out = ''.join([text for i, text in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out


# ── Extract landing status from table cell ──
def landing_status(table_cells):
    """
    Returns the landing status from the HTML table cell.
    Input: element of a table data cell.
    """
    out = [i for i in table_cells.strings][0]
    return out


# ── Extract payload mass from table cell ──
def get_mass(table_cells):
    """
    Returns the payload mass (kg) from the HTML table cell.
    """
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        new_mass = mass[0:mass.find("kg") + 2]
    else:
        new_mass = 0
    return new_mass


# ── Extract clean column name from header row ──
def extract_column_from_header(row):
    """
    Returns a cleaned column name from the header row element.
    Removes <br>, <a>, and <sup> tags.
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)

    # Filter digits and empty names
    if not colunm_name.strip().isdigit():
        colunm_name = colunm_name.strip()
        return colunm_name

To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [27]:
# ── Static Wikipedia URL for Falcon 9 and Falcon Heavy launches ──
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(static_url, headers=headers)

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [29]:
# ── Create BeautifulSoup object ──
soup = BeautifulSoup(response.text, "html.parser")

Create a `BeautifulSoup` object from the HTML `response`


In [30]:
# ── Create BeautifulSoup object from the response content ──
soup = BeautifulSoup(response.text, "html.parser")

Print the page title to verify if the `BeautifulSoup` object was created properly


In [31]:
# ── Use the title attribute of the BeautifulSoup object ──
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [32]:
# ── Buscar todos los elementos <table> y asignarlos a una lista ──
html_tables = soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [38]:
# ── Find all tables with non-empty headers ──
for i, table in enumerate(html_tables):
    headers = [th.get_text(strip=True) for th in table.find_all("th")]
    if headers:
        print(f"Table {i} headers:", headers)

launch_table = html_tables[2]  # adjust index based on the output above
column_names = [th.get_text(strip=True) for th in launch_table.find_all('th')]
print(column_names)


Table 2 headers: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']
Table 3 headers: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '8', '9', '10', '11', '12', '13']
Table 4 headers: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']
Table 5 headers: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '21', '22', '23', '24', '25', '26', '27', '28', 'N/A[e]']
Table 6 headers: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'O

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [40]:
launch_table = html_tables[2]  # use the real index from the loop above
column_names = [th.get_text(strip=True) for th in launch_table.find_all("th")]

Check the extracted column names


In [42]:
# ── Display column names in a clean format ──
launch_table = html_tables[2]  # replace 2 with the correct index you found
column_names = [th.get_text(strip=True) for th in launch_table.find_all("th")]

display(column_names)

['Flight No.',
 'Date andtime (UTC)',
 'Version,Booster[b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7']

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [44]:
# ── Initialize dictionary from column names ──
launch_dict = dict.fromkeys(column_names)

# ── Remove irrelevant column ──
del launch_dict['Date andtime (UTC)']

# ── Initialize keys with empty lists ──
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# ── Add new columns ──
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [45]:
# ── Extract rows from launch tables ──
extracted_row = 0

# ── Loop through each launch table ──
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    
    # ── Loop through each row ──
    for rows in table.find_all("tr"):
        
        # ── Check if the row starts with a numeric flight number ──
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False
        
        # ── Get all <td> cells in the row ──
        row = rows.find_all('td')
        
        # ── If valid row, extract values and append to launch_dict ──
        if flag:
            extracted_row += 1
            
            # Flight Number
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            
            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a is not None else row[6].string
            launch_dict['Customer'].append(customer)
            
            # Launch Outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# ── Convert to DataFrame for professional display ──
launch_df = pd.DataFrame(launch_dict)
display(launch_df.head(10))


,Flight No.,"Version,Booster[b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,1,...,4,5,6,7,Payload,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,None,CCAFS,None,0,LEO,SpaceX,None,None,None,...,None,None,None,None,Dragon Spacecraft Qualification Unit,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,None,CCAFS,None,0,LEO,NASA,None,None,None,...,None,None,None,None,Dragon,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,None,CCAFS,None,525 kg,LEO,NASA,None,None,None,...,None,None,None,None,Dragon,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,None,CCAFS,None,"4,700 kg",LEO,NASA,None,None,None,...,None,None,None,None,SpaceX CRS-1,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,None,CCAFS,None,"4,877 kg",LEO,NASA,None,None,None,...,None,None,None,None,SpaceX CRS-2,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10
5,6,None,VAFB,None,500 kg,Polar orbit,MDA,None,None,None,...,None,None,None,None,CASSIOPE,Success,F9 v1.17B10038,Uncontrolled,29 September 2013,16:00
6,7,None,CCAFS,None,"3,170 kg",GTO,SES,None,None,None,...,None,None,None,None,SES-8,Success,F9 v1.1,No attempt,3 December 2013,22:41
7,8,None,CCAFS,None,"3,325 kg",GTO,Thaicom,None,None,None,...,None,None,None,None,Thaicom 6,Success,F9 v1.1,No attempt,6 January 2014,22:06
8,9,None,Cape Canaveral,None,"2,296 kg",LEO,NASA,None,None,None,...,None,None,None,None,SpaceX CRS-3,Success\n,F9 v1.1,Controlled,18 April 2014,19:25
9,10,None,Cape Canaveral,None,"1,316 kg",LEO,Orbcomm,None,None,None,...,None,None,None,None,Orbcomm-OG2,Success,F9 v1.1,Controlled,14 July 2014,15:15


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [47]:
# ── Create DataFrame from launch_dict and display first rows ──
df = pd.DataFrame(launch_dict)
display(df.head())

,Flight No.,"Version,Booster[b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,1,...,4,5,6,7,Payload,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,None,CCAFS,None,0,LEO,SpaceX,None,None,None,...,None,None,None,None,Dragon Spacecraft Qualification Unit,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,None,CCAFS,None,0,LEO,NASA,None,None,None,...,None,None,None,None,Dragon,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,None,CCAFS,None,525 kg,LEO,NASA,None,None,None,...,None,None,None,None,Dragon,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,None,CCAFS,None,"4,700 kg",LEO,NASA,None,None,None,...,None,None,None,None,SpaceX CRS-1,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,None,CCAFS,None,"4,877 kg",LEO,NASA,None,None,None,...,None,None,None,None,SpaceX CRS-2,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates               |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
